The MAG data is stored in postgres `jsonb` format as described below:

In [140]:
import pandas as pd
import os
from sqlalchemy import create_engine
from sqlalchemy.sql import text as sql_text
from psycopg2.extras import execute_batch
import psycopg2
done_files = set()

In [132]:
top_dir = "data/"
argslist = []
idslist = []
for file_name in os.listdir(top_dir):
    if file_name in done_files:
        continue
    if not (file_name.startswith("arxiv") and file_name.endswith(".json")):
        continue
    done_files.add(file_name)
    _df = pd.read_json(top_dir+file_name,orient="records")    
    for _, row in _df.iterrows():
        title = "".join(row["raw_title"].split("\n "))
        title = title.replace("'","''")
        title = title.replace('"','""')
        title = title.replace("%","%%")
        idslist.append(row['id'])
        argslist.append(title)

In [141]:
with open('/Users/jklinger/Nesta-AWS/AWS-RDS-config/open-academic-graph.config') as f:
    host, port, database, user, password = f.read().split(':')
database_uri = 'postgresql://{}:{}@{}/{}'.format(user, password, host, "microsoft_academic_graph")
con = create_engine(database_uri)
#con = psycopg2.connect(database_uri)
#cur = con.cursor()
#query = '''select id from microsoft_academic_graph where ((((paper ->> 'title'::text))::character varying(300))) = %s'''

In [142]:
query = '''select id, paper ->> 'title'
from microsoft_academic_graph
where ((((paper ->> 'title'::text))::character varying(300)))
in :titles;
'''

# Stolen from https://stackoverflow.com/a/434328/1571593
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [143]:
output = []
for arg_chunk, id_chunk in zip(chunker(argslist, 100), chunker(idslist, 100)):
    exec_result = con.execute(sql_text(query), titles=tuple(arg_chunk))
    for mag_id, title in exec_result.fetchall():
        idx = arg_chunk.index(title)
        output.append(dict(arxiv_id=id_chunk[idx], mag_id=mag_id))

In [144]:
pd.DataFrame(output).to_json("data/mag_arxiv_match.json", orient="records")

In [145]:
len(output)

832104